# टेक्स्ट वर्गीकरण कार्य

जैसा कि हमने उल्लेख किया है, हम **AG_NEWS** डेटासेट पर आधारित एक सरल टेक्स्ट वर्गीकरण कार्य पर ध्यान केंद्रित करेंगे, जिसमें समाचार सुर्खियों को 4 श्रेणियों में वर्गीकृत करना है: विश्व, खेल, व्यवसाय और विज्ञान/तकनीक।

## डेटासेट

यह डेटासेट [`torchtext`](https://github.com/pytorch/text) मॉड्यूल में शामिल है, इसलिए हम इसे आसानी से एक्सेस कर सकते हैं।


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

यहाँ, `train_dataset` और `test_dataset` में संग्रह होते हैं जो क्रमशः वर्ग (कक्षा की संख्या) और पाठ के जोड़े लौटाते हैं, उदाहरण के लिए:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

तो, चलिए हमारे डेटासेट से पहले 10 नई सुर्खियाँ प्रिंट करते हैं:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

क्योंकि डेटासेट इटरेटर होते हैं, यदि हम डेटा का उपयोग कई बार करना चाहते हैं तो हमें इसे सूची में बदलना होगा:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## टोकनाइज़ेशन

अब हमें टेक्स्ट को **संख्याओं** में बदलने की आवश्यकता है, जिन्हें टेन्सर के रूप में प्रस्तुत किया जा सके। यदि हम शब्द-स्तरीय प्रतिनिधित्व चाहते हैं, तो हमें दो चीजें करनी होंगी:
* **टोकनाइज़र** का उपयोग करके टेक्स्ट को **टोकन** में विभाजित करना
* उन टोकन का एक **शब्दकोश** बनाना।


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

शब्दावली का उपयोग करके, हम आसानी से अपने टोकनयुक्त स्ट्रिंग को संख्याओं के सेट में एन्कोड कर सकते हैं:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## शब्दों का थैला (Bag of Words) टेक्स्ट प्रतिनिधित्व

क्योंकि शब्द अर्थ को दर्शाते हैं, कभी-कभी हम केवल व्यक्तिगत शब्दों को देखकर, उनके वाक्य में क्रम की परवाह किए बिना, टेक्स्ट का अर्थ समझ सकते हैं। उदाहरण के लिए, जब समाचार वर्गीकृत कर रहे हों, तो *मौसम*, *बर्फ* जैसे शब्द *मौसम पूर्वानुमान* का संकेत दे सकते हैं, जबकि *शेयर*, *डॉलर* जैसे शब्द *वित्तीय समाचार* की ओर इशारा करेंगे।

**शब्दों का थैला** (BoW) वेक्टर प्रतिनिधित्व सबसे सामान्य रूप से उपयोग किया जाने वाला पारंपरिक वेक्टर प्रतिनिधित्व है। प्रत्येक शब्द को एक वेक्टर इंडेक्स से जोड़ा जाता है, और वेक्टर तत्व में किसी दिए गए दस्तावेज़ में शब्द की घटनाओं की संख्या होती है।

![यह छवि दिखाती है कि शब्दों के थैले का वेक्टर प्रतिनिधित्व मेमोरी में कैसे दर्शाया जाता है।](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png)

> **Note**: आप BoW को टेक्स्ट में व्यक्तिगत शब्दों के लिए सभी वन-हॉट-एन्कोडेड वेक्टर का योग भी मान सकते हैं।

नीचे Scikit Learn पायथन लाइब्रेरी का उपयोग करके शब्दों के थैले का प्रतिनिधित्व बनाने का एक उदाहरण दिया गया है:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS डेटासेट के वेक्टर प्रतिनिधित्व से बैग-ऑफ-वर्ड्स वेक्टर की गणना करने के लिए, हम निम्नलिखित फ़ंक्शन का उपयोग कर सकते हैं:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **नोट:** यहां हम वैश्विक `vocab_size` चर का उपयोग कर रहे हैं ताकि शब्दावली के डिफ़ॉल्ट आकार को निर्दिष्ट किया जा सके। चूंकि अक्सर शब्दावली का आकार काफी बड़ा होता है, हम सबसे अधिक बार उपयोग किए जाने वाले शब्दों तक शब्दावली के आकार को सीमित कर सकते हैं। `vocab_size` मान को कम करने और नीचे दिए गए कोड को चलाने का प्रयास करें, और देखें कि यह सटीकता को कैसे प्रभावित करता है। आपको कुछ सटीकता में गिरावट की उम्मीद करनी चाहिए, लेकिन प्रदर्शन में सुधार के बदले यह नाटकीय नहीं होगा।


## BoW क्लासिफायर का प्रशिक्षण

अब जब हमने अपने टेक्स्ट का बैग-ऑफ-वर्ड्स प्रतिनिधित्व बनाना सीख लिया है, तो आइए इसके ऊपर एक क्लासिफायर को प्रशिक्षित करें। सबसे पहले, हमें अपने डेटासेट को इस तरह से प्रशिक्षण के लिए बदलने की आवश्यकता है कि सभी पोजिशनल वेक्टर प्रतिनिधित्व को बैग-ऑफ-वर्ड्स प्रतिनिधित्व में परिवर्तित किया जा सके। इसे `bowify` फ़ंक्शन को मानक टॉर्च `DataLoader` में `collate_fn` पैरामीटर के रूप में पास करके प्राप्त किया जा सकता है:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

अब चलिए एक साधारण वर्गीकरण न्यूरल नेटवर्क को परिभाषित करते हैं जिसमें एक रैखिक परत होती है। इनपुट वेक्टर का आकार `vocab_size` के बराबर है, और आउटपुट आकार वर्गों की संख्या (4) के अनुरूप है। क्योंकि हम वर्गीकरण कार्य को हल कर रहे हैं, अंतिम सक्रियता फ़ंक्शन `LogSoftmax()` है।


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

अब हम मानक PyTorch प्रशिक्षण लूप को परिभाषित करेंगे। क्योंकि हमारा डेटासेट काफी बड़ा है, शिक्षण उद्देश्य के लिए हम केवल एक युग के लिए प्रशिक्षण करेंगे, और कभी-कभी एक युग से भी कम (प्रशिक्षण को सीमित करने के लिए `epoch_size` पैरामीटर निर्दिष्ट किया जाता है)। हम प्रशिक्षण के दौरान संचित प्रशिक्षण सटीकता की भी रिपोर्ट करेंगे; रिपोर्टिंग की आवृत्ति `report_freq` पैरामीटर का उपयोग करके निर्दिष्ट की जाती है।


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## बाईग्राम्स, ट्राईग्राम्स और एन-ग्राम्स

बैग ऑफ वर्ड्स दृष्टिकोण की एक सीमा यह है कि कुछ शब्द बहु-शब्द अभिव्यक्तियों का हिस्सा होते हैं। उदाहरण के लिए, 'हॉट डॉग' शब्द का अर्थ पूरी तरह से अलग होता है, जबकि 'हॉट' और 'डॉग' शब्द अन्य संदर्भों में अलग-अलग अर्थ रखते हैं। यदि हम हमेशा 'हॉट' और 'डॉग' शब्दों को एक ही वेक्टर द्वारा प्रदर्शित करें, तो यह हमारे मॉडल को भ्रमित कर सकता है।

इस समस्या को हल करने के लिए, **एन-ग्राम प्रतिनिधित्व** का उपयोग अक्सर दस्तावेज़ वर्गीकरण की विधियों में किया जाता है, जहां प्रत्येक शब्द, द्वि-शब्द या त्रि-शब्द की आवृत्ति वर्गीकरणकर्ताओं को प्रशिक्षित करने के लिए एक उपयोगी विशेषता होती है। उदाहरण के लिए, बाईग्राम प्रतिनिधित्व में, हम मूल शब्दों के अलावा सभी शब्द युग्मों को शब्दावली में जोड़ेंगे।

नीचे एक उदाहरण दिया गया है कि कैसे Scikit Learn का उपयोग करके बाईग्राम बैग ऑफ वर्ड्स प्रतिनिधित्व बनाया जा सकता है:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram दृष्टिकोण की मुख्य कमी यह है कि शब्दावली का आकार बहुत तेजी से बढ़ने लगता है। व्यवहार में, हमें N-gram प्रतिनिधित्व को कुछ आयाम-घटाने की तकनीकों, जैसे *embeddings*, के साथ संयोजित करने की आवश्यकता होती है, जिन पर हम अगले यूनिट में चर्चा करेंगे।

हमारे **AG News** डेटासेट में N-gram प्रतिनिधित्व का उपयोग करने के लिए, हमें एक विशेष ngram शब्दावली बनानी होगी:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


हम ऊपर दिए गए कोड का उपयोग करके क्लासिफायर को ट्रेन कर सकते हैं, लेकिन यह मेमोरी के लिहाज से बहुत अक्षम होगा। अगले यूनिट में, हम एम्बेडिंग्स का उपयोग करके बिग्राम क्लासिफायर को ट्रेन करेंगे।

> **नोट:** आप केवल उन्हीं ngrams को छोड़ सकते हैं जो टेक्स्ट में निर्दिष्ट संख्या से अधिक बार आते हैं। यह सुनिश्चित करेगा कि कम बार आने वाले बिग्राम्स को हटा दिया जाए, और डाइमेंशनलिटी को काफी हद तक कम किया जा सके। ऐसा करने के लिए, `min_freq` पैरामीटर को उच्च मान पर सेट करें, और शब्दावली की लंबाई में बदलाव को देखें।


## टर्म फ्रीक्वेंसी इनवर्स डॉक्यूमेंट फ्रीक्वेंसी (TF-IDF)

BoW (बैग ऑफ वर्ड्स) प्रतिनिधित्व में, शब्दों की उपस्थिति को समान रूप से महत्व दिया जाता है, चाहे वह शब्द कोई भी हो। हालांकि, यह स्पष्ट है कि सामान्य शब्द जैसे *a*, *in* आदि वर्गीकरण के लिए उतने महत्वपूर्ण नहीं होते जितने कि विशेष शब्द। वास्तव में, अधिकांश NLP कार्यों में कुछ शब्द अन्य शब्दों की तुलना में अधिक प्रासंगिक होते हैं।

**TF-IDF** का मतलब है **टर्म फ्रीक्वेंसी–इनवर्स डॉक्यूमेंट फ्रीक्वेंसी**। यह बैग ऑफ वर्ड्स का एक प्रकार है, जिसमें किसी दस्तावेज़ में शब्द की उपस्थिति को दर्शाने वाले 0/1 बाइनरी मान के बजाय एक फ्लोटिंग-पॉइंट मान का उपयोग किया जाता है, जो कॉर्पस में शब्द की उपस्थिति की आवृत्ति से संबंधित होता है।

औपचारिक रूप से, किसी शब्द $i$ का वजन $w_{ij}$ किसी दस्तावेज़ $j$ में इस प्रकार परिभाषित किया जाता है:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
जहां:
* $tf_{ij}$ किसी दस्तावेज़ $j$ में शब्द $i$ की उपस्थिति की संख्या है, यानी वह BoW मान जिसे हमने पहले देखा था
* $N$ संग्रह में दस्तावेज़ों की कुल संख्या है
* $df_i$ पूरे संग्रह में शब्द $i$ को शामिल करने वाले दस्तावेज़ों की संख्या है

TF-IDF मान $w_{ij}$ किसी दस्तावेज़ में शब्द की उपस्थिति की संख्या के अनुपात में बढ़ता है और कॉर्पस में उस शब्द को शामिल करने वाले दस्तावेज़ों की संख्या से समायोजित होता है। यह इस तथ्य को संतुलित करने में मदद करता है कि कुछ शब्द अन्य शब्दों की तुलना में अधिक बार दिखाई देते हैं। उदाहरण के लिए, यदि कोई शब्द *हर* दस्तावेज़ में दिखाई देता है, तो $df_i=N$, और $w_{ij}=0$, और ऐसे शब्दों को पूरी तरह से नजरअंदाज कर दिया जाएगा।

आप आसानी से Scikit Learn का उपयोग करके टेक्स्ट का TF-IDF वेक्टराइज़ेशन बना सकते हैं:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## निष्कर्ष

हालांकि TF-IDF प्रतिनिधित्व विभिन्न शब्दों को आवृत्ति भार प्रदान करते हैं, वे अर्थ या क्रम को व्यक्त करने में असमर्थ होते हैं। जैसा कि प्रसिद्ध भाषाविद् जे. आर. फर्थ ने 1935 में कहा था, "शब्द का पूर्ण अर्थ हमेशा संदर्भात्मक होता है, और संदर्भ से अलग अर्थ का कोई भी अध्ययन गंभीरता से नहीं लिया जा सकता।" हम इस पाठ्यक्रम में आगे सीखेंगे कि भाषा मॉडलिंग का उपयोग करके पाठ से संदर्भात्मक जानकारी कैसे प्राप्त करें।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
